In [16]:
import matplotlib.pyplot as plt
import pandas as pd

from markovBike.manager.manager import Manager
from markovBike.data_source.source import database_queries, get_stations_data

import os
import requests

In [17]:
def calculate_daily_rides(dataframe):
    # Convert the starttime column to a pandas datetime object
    dataframe['starttime'] = pd.to_datetime(dataframe['starttime'])

    # Add a new column with just the date portion of the starttime column
    dataframe['date'] = dataframe['starttime'].dt.date

    # Count the number of rows (i.e. bike rides) per day
    daily_rides = dataframe.groupby('date').size().reset_index(
        name='daily_rides')

    return daily_rides

In [18]:
def get_weather_data(date):
    # Replace YOUR_API_KEY with your own OpenWeatherMap API key
    api_key = os.environ.get('API_KEY')
    url = f"https://api.openweathermap.org/data/2.5/weather?q=New%20York&units=imperial&appid={api_key}&dt={date}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather_data = {
            "date": date,
            "temperature": data["main"]["temp"],
            "description": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"]
        }
        print(f'Weather for {date} requested', end='\r')
        return weather_data
    else:
        print(f"Error: {response.status_code}", end='\n')


In [19]:
def get_daily_rides_with_weather(df):
    df['weather'] = df['date'].apply(get_weather_data)

    # Unpack weather dictionary into separate columns
    df['temperature'] = df['weather'].apply(lambda x: x['temperature'])
    df['description'] = df['weather'].apply(lambda x: x['description'])
    df['humidity'] = df['weather'].apply(lambda x: x['humidity'])
    df['wind_speed'] = df['weather'].apply(lambda x: x['wind_speed'])

    # Drop the 'weather' column
    df.drop('weather', axis=1, inplace=True)

    return df


In [20]:
n_trips = 10

trips_raw = get_stations_data(database_queries(n_trips)['trips'], verbose=True)

trips_raw['date'] = pd.to_datetime(trips_raw['starttime']).dt.date

daily_rides = calculate_daily_rides(trips_raw)

merged_df = trips_raw.merge(daily_rides, on='date')

weather_dataframe = get_daily_rides_with_weather(merged_df)

Bike station table with shape (10, 15). Columns are: 

tripduration                             Int64
starttime                  datetime64[ns, UTC]
stoptime                   datetime64[ns, UTC]
start_station_id                         Int64
start_station_name                      object
start_station_latitude                 float64
start_station_longitude                float64
end_station_id                           Int64
end_station_name                        object
end_station_latitude                   float64
end_station_longitude                  float64
bikeid                                   Int64
usertype                                object
birth_year                               Int64
gender                                  object
dtype: object



In [21]:
weather_dataframe

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,...,bikeid,usertype,birth_year,gender,date,daily_rides,temperature,description,humidity,wind_speed
0,107,2014-06-06 13:02:01+00:00,2014-06-06 13:03:48+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,16692,Subscriber,1977,male,2014-06-06,1,55.96,few clouds,60,12.66
1,1105,2013-08-24 17:03:52+00:00,2013-08-24 17:22:17+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,15352,Customer,<NA>,unknown,2013-08-24,1,55.99,few clouds,60,12.66
2,990,2014-04-25 11:52:28+00:00,2014-04-25 12:08:58+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,20194,Subscriber,1973,male,2014-04-25,1,55.96,few clouds,60,12.66
3,138,2015-08-20 16:58:57+00:00,2015-08-20 17:01:16+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,21002,Subscriber,1977,male,2015-08-20,1,55.99,few clouds,60,12.66
4,78,2014-09-13 13:18:36+00:00,2014-09-13 13:19:54+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,19259,Customer,<NA>,unknown,2014-09-13,1,55.99,few clouds,60,12.66
5,826,2015-08-26 19:48:32+00:00,2015-08-26 20:02:19+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,17766,Customer,<NA>,unknown,2015-08-26,1,56.08,few clouds,60,12.66
6,122,2015-11-30 16:20:57+00:00,2015-11-30 16:23:00+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,17913,Customer,<NA>,unknown,2015-11-30,1,55.96,few clouds,60,12.66
7,999,2015-10-30 12:58:52+00:00,2015-10-30 13:15:32+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,23325,Subscriber,1958,male,2015-10-30,1,56.03,few clouds,60,12.66
8,1745,2014-04-03 11:57:26+00:00,2014-04-03 12:26:31+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,21445,Subscriber,1979,male,2014-04-03,1,55.96,few clouds,60,12.66
9,128,2016-08-29 13:48:25+00:00,2016-08-29 13:50:33+00:00,520,W 52 St & 5 Ave,40.759923,-73.976485,520,W 52 St & 5 Ave,40.759923,...,23801,Customer,<NA>,unknown,2016-08-29,1,55.96,few clouds,60,12.66
